In [322]:
%pip install numpy numba tqdm pandas
%pip install --no-cache-dir --force-reinstall https://dm.cs.tu-dortmund.de/nats/nats25_02_02_bpe-0.1-py3-none-any.whl
import nats25_02_02_bpe

15111.84s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


15117.51s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


  Attempting uninstall: nats25-02-02-bpe
    Found existing installation: nats25_02_02_bpe 0.1
    Uninstalling nats25_02_02_bpe-0.1:
      Successfully uninstalled nats25_02_02_bpe-0.1
Note: you may need to restart the kernel to use updated packages.


# Byte-Pair Encoding

In this assignment, your task is to implement the training of a byte-pair-encoding tokenizer yourself.

In [323]:
import numpy as np, pandas as pd, re
from numba import jit
try: from tqdm.notebook import tqdm # optional
except: tqdm = None

# Load the input data
import gzip, json, urllib
file_path, _ = urllib.request.urlretrieve("https://dm.cs.tu-dortmund.de/nats/data/minecraft-articles.json.gz")
raw = json.load(gzip.open(file_path, "rt", encoding="utf-8"))
titles, texts, classes = [x["title"] for x in raw], [x["text"] for x in raw], [x["heuristic"] for x in raw]

## Join texts into a single sequence of bytes.

Split all the provided texts (`title` and `text`) using the given whitespace pretokenizer. Encode the tokens as bytes with UTF-8.

In [ ]:
#in this we tokenized the titles and texts into one singular data array where each element is a byte represenation of each token 
pretokenizer=re.compile(r"\n|\s*\S+")
data = []
for title in titles:
    split_text = pretokenizer.findall(string=title)
    for elm in split_text:
        data.append(elm)

for text in texts: 
    split_text = pretokenizer.findall(string=text)
    for elm in split_text:
        data.append(elm)
for index in range(0,len(data)): 
    data[index] = data[index].encode(encoding="utf-8")
print(len(data))


714539


In [325]:
nats25_02_02_bpe.hidden_tests_4_0(data)

In [326]:
# In the following, we will use lists containing numpy arrays with int16
data = np.array([int(x) for x in b"\0".join(data)], dtype=np.int16)
print(data.shape)

(4649344,)


In [327]:
nats25_02_02_bpe.hidden_tests_6_0(data)

## Write a function to find the most common two symbols in a sequence

While this will be the performance bottleneck of the implementation, you may use a `Counter` of pairs here.

In our experiments, a vectorized numpy solution was 60x faster.

- Return a pair of ints (we *will* exceed the byte range).
- Skip 0 tokens used as separators
- The second token must not be a space or newline ("pre-tokenization")
- When no token occurs more than once, return None

In [328]:
from collections import Counter
def find_most_frequent(seq):
    """
    Alternative vectorized solution using hash-based approach for integer sequences.
    This is faster for integer sequences but limited to integer data types.
    
    Args:
        seq: sequence/array of integers
    
    Returns:
        tuple: most frequent pair or None if no pair occurs more than once
    """
    seq = np.asarray(seq)
    
    if len(seq) < 2:
        return None
    
    # Create adjacent pairs
    pairs_left = seq[:-1]
    pairs_right = seq[1:]
    
    # Filter out pairs containing 0
    mask = (pairs_left != 0) & (pairs_right != 0)
    
    if not np.any(mask):
        return None
    
    valid_left = pairs_left[mask]
    valid_right = pairs_right[mask]
    
    if len(valid_left) == 0:
        return None
    
    # For integer sequences, we can use a hash-like approach
    # Combine pairs into single numbers for faster counting
    max_val = max(np.max(valid_left), np.max(valid_right))
    if max_val < 2**16:  # Safe for combining into 32-bit int
        combined = valid_left.astype(np.int32) * (max_val + 1) + valid_right.astype(np.int32)
        unique_combined, counts = np.unique(combined, return_counts=True)
        
        max_count_idx = np.argmax(counts)
        max_count = counts[max_count_idx]
        
        if max_count > 1:
            most_frequent_combined = unique_combined[max_count_idx]
            left = most_frequent_combined // (max_val + 1)
            right = most_frequent_combined % (max_val + 1)
            return (np.int16(left), np.int16(right))
    
    return None
'''def find_most_frequent(seq):
    # We create pairs of adjacent tokens from the sequence.
    # We filter out any pairs that contain the '0' separator token,
    # as instructed.
    pairs = (
        (seq[i], seq[i+1]) 
        for i in range(len(seq) - 1) 
        if seq[i] != 0 and seq[i+1] != 0
    )
    
    # We use Counter to find the most common pair.
    pair_counts = Counter(pairs)
    
    # If the counter is empty or no pair occurs more than once, return None.
    if not pair_counts:
        return None
        
    most_frequent_pair, count = pair_counts.most_common(1)[0]
    
    if count > 1:
        return most_frequent_pair
    else:
        return None
'''
'''def find_most_frequent1(seq):
    df = pd.DataFrame(seq, columns=['value'])
    df = df[df.value != 0]
    seq = df.to_numpy()
    seq = seq.squeeze()
    c = Counter(seq)
    most_freq = c.most_common(n=2)
    print("most freq element: ", most_freq[0][0])
    return most_freq[0][0], most_freq[1][0]
'''
result = find_most_frequent(seq = data)
x,y = result 
print(x,y)

32 116


In [329]:
nats25_02_02_bpe.hidden_tests_9_0(find_most_frequent, data)

## Initialize the vocabulary

Our initial vocabulary contains all 256 bytes, so we can later still encode any character (or byte sequence) not in our training data.
The vocabulary is used for decoding, so it is a map from integer token ids to bytes.

In [330]:
def init_vocab():
    #Encode all of the integers 0 to 255 to their byte representations
    #the integers in this case represent the ascii code for the symbol 
    vocab = dict()
    for i in range(0,256): 
        vocab[i] = bytes([i])
    return vocab
init_vocab()

{0: b'\x00',
 1: b'\x01',
 2: b'\x02',
 3: b'\x03',
 4: b'\x04',
 5: b'\x05',
 6: b'\x06',
 7: b'\x07',
 8: b'\x08',
 9: b'\t',
 10: b'\n',
 11: b'\x0b',
 12: b'\x0c',
 13: b'\r',
 14: b'\x0e',
 15: b'\x0f',
 16: b'\x10',
 17: b'\x11',
 18: b'\x12',
 19: b'\x13',
 20: b'\x14',
 21: b'\x15',
 22: b'\x16',
 23: b'\x17',
 24: b'\x18',
 25: b'\x19',
 26: b'\x1a',
 27: b'\x1b',
 28: b'\x1c',
 29: b'\x1d',
 30: b'\x1e',
 31: b'\x1f',
 32: b' ',
 33: b'!',
 34: b'"',
 35: b'#',
 36: b'$',
 37: b'%',
 38: b'&',
 39: b"'",
 40: b'(',
 41: b')',
 42: b'*',
 43: b'+',
 44: b',',
 45: b'-',
 46: b'.',
 47: b'/',
 48: b'0',
 49: b'1',
 50: b'2',
 51: b'3',
 52: b'4',
 53: b'5',
 54: b'6',
 55: b'7',
 56: b'8',
 57: b'9',
 58: b':',
 59: b';',
 60: b'<',
 61: b'=',
 62: b'>',
 63: b'?',
 64: b'@',
 65: b'A',
 66: b'B',
 67: b'C',
 68: b'D',
 69: b'E',
 70: b'F',
 71: b'G',
 72: b'H',
 73: b'I',
 74: b'J',
 75: b'K',
 76: b'L',
 77: b'M',
 78: b'N',
 79: b'O',
 80: b'P',
 81: b'Q',
 82: b'R',
 83: b'

In [331]:
nats25_02_02_bpe.hidden_tests_12_0(init_vocab)

## Token replacement function

In the given sequence, replace tokens (a,b) with a new token c. Avoid copying, but modify the sequence in-place. You can use `numba.jit` to make this (much) faster.

Return the resulting array (-view).

In [332]:
@jit(nopython=True)
def replace(seq, a, b, c):
    """
    A highly optimized Numba version that avoids list appends by using a 
    pre-allocated NumPy array.
    """
    # Create a result array of the same type and maximum possible size.
    new_seq = np.empty_like(seq)
    
    # 'i' will be the read-index for the input sequence 'seq'
    # 'j' will be the write-index for the output sequence 'new_seq'
    i = 0
    j = 0 
    
    while i < len(seq):
        if i + 1 < len(seq) and seq[i] == a and seq[i+1] == b:
            new_seq[j] = c
            i += 2 # Advance read-index by 2
            j += 1 # Advance write-index by 1
        else:
            new_seq[j] = seq[i]
            i += 1 # Advance read-index by 1
            j += 1 # Advance write-index by 1
    
    # The final result is the slice of the array that we actually used.
    # This trimming operation is very fast.
    return new_seq[:j]

In [333]:
nats25_02_02_bpe.hidden_tests_15_0(replace)

## Train BPE

Implement a function to train a byte-pair encoding.

In [346]:
def train_bpe(indata, size=1000):
    merges = list() # of tuples(id1, id2)
    new_id = 256
    vocab = init_vocab()
    data = np.array(indata, dtype=np.int16) # copy to allow modifications
    print("length of data: ", len(data))
    print(data)
    #scan the text to find the most common pairs 
    while len(vocab) < size: 
        pair = find_most_frequent(data)
        data = replace(data, pair[0], pair[1], new_id)
        new_id = len(vocab)
        #add the mapping such that it can be reversed (decoded)
        vocab[new_id] = vocab[pair[0]] + vocab[pair[1]]
        merges.append((pair,new_id))
    print(vocab)
    print("Compression factor:", len(data) / len(indata))
    return vocab, merges


In [335]:
nats25_02_02_bpe.hidden_tests_18_0(train_bpe)

length of data:  29
[ 72 101 108 108 111  32 119 111 114 108 100  44  32 104 101 108 108 111
  32 101 118 101 114 121  98 111 100 121  46]
{0: b'\x00', 1: b'\x01', 2: b'\x02', 3: b'\x03', 4: b'\x04', 5: b'\x05', 6: b'\x06', 7: b'\x07', 8: b'\x08', 9: b'\t', 10: b'\n', 11: b'\x0b', 12: b'\x0c', 13: b'\r', 14: b'\x0e', 15: b'\x0f', 16: b'\x10', 17: b'\x11', 18: b'\x12', 19: b'\x13', 20: b'\x14', 21: b'\x15', 22: b'\x16', 23: b'\x17', 24: b'\x18', 25: b'\x19', 26: b'\x1a', 27: b'\x1b', 28: b'\x1c', 29: b'\x1d', 30: b'\x1e', 31: b'\x1f', 32: b' ', 33: b'!', 34: b'"', 35: b'#', 36: b'$', 37: b'%', 38: b'&', 39: b"'", 40: b'(', 41: b')', 42: b'*', 43: b'+', 44: b',', 45: b'-', 46: b'.', 47: b'/', 48: b'0', 49: b'1', 50: b'2', 51: b'3', 52: b'4', 53: b'5', 54: b'6', 55: b'7', 56: b'8', 57: b'9', 58: b':', 59: b';', 60: b'<', 61: b'=', 62: b'>', 63: b'?', 64: b'@', 65: b'A', 66: b'B', 67: b'C', 68: b'D', 69: b'E', 70: b'F', 71: b'G', 72: b'H', 73: b'I', 74: b'J', 75: b'K', 76: b'L', 77: b'M', 

## Train a tokenizer on our training data

Inspect the longest tokens generated.

In [349]:
%%time
vocab, merges = train_bpe(data, 512) # begin with 512 – at 1024, we get many more words as standalone tokens, but the runtime increases

length of data:  4649344
[ 83 101 114 ... 109 111 114]
{0: b'\x00', 1: b'\x01', 2: b'\x02', 3: b'\x03', 4: b'\x04', 5: b'\x05', 6: b'\x06', 7: b'\x07', 8: b'\x08', 9: b'\t', 10: b'\n', 11: b'\x0b', 12: b'\x0c', 13: b'\r', 14: b'\x0e', 15: b'\x0f', 16: b'\x10', 17: b'\x11', 18: b'\x12', 19: b'\x13', 20: b'\x14', 21: b'\x15', 22: b'\x16', 23: b'\x17', 24: b'\x18', 25: b'\x19', 26: b'\x1a', 27: b'\x1b', 28: b'\x1c', 29: b'\x1d', 30: b'\x1e', 31: b'\x1f', 32: b' ', 33: b'!', 34: b'"', 35: b'#', 36: b'$', 37: b'%', 38: b'&', 39: b"'", 40: b'(', 41: b')', 42: b'*', 43: b'+', 44: b',', 45: b'-', 46: b'.', 47: b'/', 48: b'0', 49: b'1', 50: b'2', 51: b'3', 52: b'4', 53: b'5', 54: b'6', 55: b'7', 56: b'8', 57: b'9', 58: b':', 59: b';', 60: b'<', 61: b'=', 62: b'>', 63: b'?', 64: b'@', 65: b'A', 66: b'B', 67: b'C', 68: b'D', 69: b'E', 70: b'F', 71: b'G', 72: b'H', 73: b'I', 74: b'J', 75: b'K', 76: b'L', 77: b'M', 78: b'N', 79: b'O', 80: b'P', 81: b'Q', 82: b'R', 83: b'S', 84: b'T', 85: b'U', 86: 

In [341]:
nats25_02_02_bpe.hidden_tests_21_0(vocab, merges)

AssertionError: No token for ' block'?

## Tokenization function

Implement a function to tokenize a string given the vocabulary and merges.

While not the most efficient, it is fine to implement this using `replace` above. To improve performance, call `replace` only when necessary.

In [ ]:
def tokenize(merges, s):
    #in this we tokenized the titles and texts into one singular data array where each element is a byte represenation of each token 
    pretokenizer=re.compile(r"\n|\s*\S+")
    data = []
    for words in s:
        split_text = pretokenizer.findall(string=title)
        for elm in split_text:
            data.append(elm)
    for index in range(0,len(data)): 
        data[index] = data[index].encode(encoding="utf-8")
    print(len(data))
    # In the following, we will use lists containing numpy arrays with int16
    data = np.array([int(x) for x in b"\0".join(data)], dtype=np.int16)
    while len(vocab) < size: 
        pair = find_most_frequent(data)
        data = replace(data, pair[0], pair[1], new_id)
        new_id = len(vocab)
        #add the mapping such that it can be reversed (decoded)
        vocab[new_id] = vocab[pair[0]] + vocab[pair[1]]
        merges.append((pair,new_id))

s_tmp = tokenize(merges, "The data set is about Minecraft.")


32


In [345]:
nats25_02_02_bpe.hidden_tests_24_0(vocab, merges, tokenize)

TypeError: object of type 'NoneType' has no len()

## Decoding function

Implement a function to decode a token sequence into a regular string.

In [ ]:
def decode(vocab, tokens):
    s = None
    pass # Your solution here
    return s

In [ ]:
nats25_02_02_bpe.hidden_tests_27_0(vocab, merges, tokenize, decode)